Block 0 — Install

In [ ]:
%pip install -U -r ../env/requirements.txt
# Restart kernel after install

Block 1 — System & project check 

In [ ]:
# System + project check (adds project root to imports)
import sys, platform, multiprocessing, psutil
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
Path(PROJECT_ROOT / "src" / "__init__.py").touch()

# Settings
BASE_PATH = Path("/Volumes/My Passport/")   # <- change to your audio root
PATTERN   = "AudioMoth_*"                   # or None
OUT_DIR   = (PROJECT_ROOT / "results" / "csv").resolve()
CONF      = 0.10
CPU_CORES = multiprocessing.cpu_count()
RECOMMENDED_WORKERS = max(1, CPU_CORES - 1)  # best default on CPU
WORKERS   = 2 

OUT_DIR.mkdir(parents=True, exist_ok=True)

# Diagnostics (short)
print("— System —")
print("OS:", platform.system(), platform.release(), "| Py:", platform.python_version())
print("Arch:", platform.machine(), "| CPU cores:", CPU_CORES)
try:
    import tensorflow as tf
    print("TF:", tf.__version__, "(lite available)")
except Exception:
    try:
        import tflite_runtime.interpreter as _tfl
        print("tflite_runtime available")
    except Exception:
        print("TFLite: not detected (birdnetlib will raise if missing)")

vm = psutil.virtual_memory()
print("RAM:", round(vm.total/1024**3, 1), "GB")
print("GPU note: birdnetlib/TFLite uses CPU (XNNPACK), not GPU.")
print("OUT_DIR:", OUT_DIR)

Block 2 — Discover files

In [ ]:
from src.birdnet_batch import find_audio_files

files = find_audio_files(BASE_PATH, PATTERN)
print("Found files:", len(files))
for f in files[:10]: print("-", f)
assert files, "No audio files found."

Block 3 — Test one file → CSV

In [ ]:
from src.birdnet_batch import analyze_one_to_csv

test_file = files[0]  # choose another if you want
print("Test file:", test_file)
csv_path = analyze_one_to_csv(test_file, OUT_DIR, min_conf=CONF)
print("Wrote:", csv_path)

Block 4 — Full batch → per-file CSVs

In [4]:
# run once
import importlib, src.birdnet_batch as bb
importlib.reload(bb)

from src.birdnet_batch import analyze_batch_with_progress, compile_master_csv

In [ ]:
# Full batch with progress 
from src.birdnet_batch import analyze_batch_with_progress, compile_master_csv

written = analyze_batch_with_progress(
    files, OUT_DIR, min_conf=CONF, workers=WORKERS  # WORKERS from Block 1 (CPU_CORES-1)
)
print("Per-file CSVs:", len(written))

master = compile_master_csv(OUT_DIR, master_name="master_results.csv")
print("Master CSV:", master, "| exists:", master.exists())

Block 5 — Quick QA

In [ ]:
import pandas as pd
master = OUT_DIR / "master_results.csv"
df = pd.read_csv(master)
print(df.shape)
df.head()